<a href="https://colab.research.google.com/github/testuayun/NWU-nCov-auto-report/blob/master/Fine_tune_PaliGemma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PaliGemma Fine-tuning

In this notebook, we will fine-tune [pretrained PaliGemma](https://huggingface.co/google/paligemma2-3b-pt-448) on a small split of [VQAv2](https://huggingface.co/datasets/HuggingFaceM4/VQAv2) dataset. Let's get started by installing necessary libraries.

In [1]:
!pip install -q -U datasets bitsandbytes peft git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB

We will authenticate to access the model using `notebook_login()`.

In [4]:
from huggingface_hub import notebook_login
notebook_login()

Let's load the dataset.

In [ ]:
from datasets import load_dataset
ds = load_dataset('merve/vqav2-small', split="validation")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
split_ds = ds.train_test_split(test_size=0.9) # we'll use a very small split for demo
train_ds = split_ds["test"]

In [ ]:
train_ds

Dataset({
    features: ['multiple_choice_answer', 'question', 'image'],
    num_rows: 19292
})

Our dataset is a very general one and similar to many datasets that PaliGemma was trained with. In this case, we do not need to fine-tune the image encoder, the multimodal projector but we will only fine-tune the text decoder.

In [ ]:
from transformers import PaliGemmaProcessor
model_id ="google/paligemma-3b-ft-vqav2-448" # or your favorite PaliGemma

In [ ]:
from transformers import PaliGemmaForConditionalGeneration
import torch
device = "cuda"
model = PaliGemmaForConditionalGeneration.from_pretrained(model_id, torch_dtype=torch.bfloat16).to(device)

for param in model.vision_tower.parameters():
    param.requires_grad = False

for param in model.multi_modal_projector.parameters():
    param.requires_grad = False


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Alternatively, if you want to do LoRA and QLoRA fine-tuning, you can run below cells to load the adapter either in full precision or quantized.

In [ ]:
from transformers import BitsAndBytesConfig, PaliGemmaForConditionalGeneration
from peft import get_peft_model, LoraConfig

bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16)

lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

model = PaliGemmaForConditionalGeneration.from_pretrained(model_id, device_map="auto")#, quantization_config=bnb_config)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
#trainable params: 11,298,816 || all params: 2,934,634,224 || trainable%: 0.38501616002417344


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 11,876,352 || all params: 3,044,118,768 || trainable%: 0.3901


We need to take tokens to same dtype as model so need to store it as a variable.

In [ ]:
DTYPE = model.dtype

Load the processor to preprocess the dataset.

In [ ]:
processor = PaliGemmaProcessor.from_pretrained(model_id)

We will preprocess our examples. We need to prepare a prompt template and pass the text input inside, pass it with batches of images to processor. Then we will set the pad tokens and image tokens to -100 to let the model ignore them. We will pass our preprocessed input as labels to make the model learn how to generate responses.

In [ ]:
import torch

image_token = processor.tokenizer.convert_tokens_to_ids("<image>")
def collate_fn(examples):
  texts = ["<image>answer en " + example["question"] for example in examples]
  labels= [example['multiple_choice_answer'] for example in examples]
  images = [example["image"].convert("RGB") for example in examples]
  tokens = processor(text=texts, images=images, suffix=labels,
                    return_tensors="pt", padding="longest")

  tokens = tokens.to(DTYPE).to(device)
  return tokens


We will now initialize the `TrainingArguments`.

In [ ]:
from transformers import TrainingArguments
args=TrainingArguments(
            num_train_epochs=2,
            remove_unused_columns=False,
            per_device_train_batch_size=1,
            gradient_accumulation_steps=4,
            warmup_steps=2,
            learning_rate=2e-5,
            weight_decay=1e-6,
            adam_beta2=0.999,
            logging_steps=100,
            optim="adamw_hf", # you can use paged optimizers like paged_adamw_8bit for QLoRA
            save_strategy="steps",
            save_steps=1000,
            save_total_limit=1,
            output_dir="paligemma_vqav2",
            bf16=True,
            report_to=["tensorboard"],
            dataloader_pin_memory=False
        )


We can now start training.

In [ ]:
from transformers import Trainer

trainer = Trainer(
        model=model,
        train_dataset=train_ds ,
        data_collator=collate_fn,
        args=args
        )


LoRA with bsz of 2 works on A100 Colab. You can apply gradient accumulation (which is enabled in this notebook) to simulate larger batch sizes.
Currently there's an issue with QLoRA, we are investigating and will solve soon.

In [ ]:
trainer.train()

In [ ]:
trainer.push_to_hub()

In [ ]:
!pip install -q transformers datasets evaluate accelerate Pillow pandas
!sudo apt-get install git-lfs  # 大文件下载支持

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from PIL import Image
import torch
from transformers import PaliGemmaForConditionalGeneration, PaliGemmaProcessor
from datasets import load_dataset
from tqdm.auto import tqdm
import evaluate
import numpy as np

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model_id = "google/paligemma-3b-ft-vqav2-448"

# 优化加载配置（节省显存）
model = PaliGemmaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,  # 使用 BF16 节省显存
    device_map="auto",
    revision="bfloat16"  # 指定量化版本
)
processor = PaliGemmaProcessor.from_pretrained(model_id)

config.json:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/62.6k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/862M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

In [ ]:
# 自动从 HuggingFace 加载已处理的数据集（无需手动下载）
dataset = load_dataset('merve/vqav2-small', split="validation")


# 创建易于处理的子集（可选，全量验证需删除）
# dataset = dataset.select(range(2000))  # 测试用 500 样本

README.md:   0%|          | 0.00/403 [00:00<?, ?B/s]

validation-00000-of-00007.parquet:   0%|          | 0.00/484M [00:00<?, ?B/s]

validation-00001-of-00007.parquet:   0%|          | 0.00/486M [00:00<?, ?B/s]

validation-00002-of-00007.parquet:   0%|          | 0.00/483M [00:00<?, ?B/s]

validation-00003-of-00007.parquet:   0%|          | 0.00/486M [00:00<?, ?B/s]

validation-00004-of-00007.parquet:   0%|          | 0.00/475M [00:00<?, ?B/s]

validation-00005-of-00007.parquet:   0%|          | 0.00/484M [00:00<?, ?B/s]

validation-00006-of-00007.parquet:   0%|          | 0.00/479M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/21435 [00:00<?, ? examples/s]

In [ ]:
dataset.column_names
dataset[0]

{'multiple_choice_answer': 'carnival ride',
 'question': 'Where are the kids riding?',
 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x424>}

In [ ]:
def clean_answer(answer: str) -> str:
    return answer.strip().lower().replace(".", "").split()[0]

In [ ]:
def batch_predict(dataset, batch_size=4):
    all_answers = []

    for i in tqdm(range(0, len(dataset), batch_size)):
        batch = dataset.select(range(i, min(i+batch_size, len(dataset))))

        # 构建 prompts
        prompts = [f"answer en {ex['question']}" for ex in batch]

        # 预处理
        inputs = processor(
            text=prompts,
            images=[ex["image"].convert("RGB") for ex in batch],
            return_tensors="pt",
            padding=True,
            truncation=True
        ).to(model.device)

        # 生成答案
        outputs = model.generate(
            **inputs,
            max_new_tokens=10,
            do_sample=False,
            num_beams=3
        )

        # 解码
        batch_answers = processor.batch_decode(outputs, skip_special_tokens=True)
        all_answers.extend([clean_answer(a) for a in batch_answers])

    return all_answers

In [ ]:
predicted_answers = batch_predict(dataset, batch_size=8)

# 准备评估输入（注意简化版数据集的结构）
references = [{"answers": [ex["answer"]]} for ex in dataset]  # 单答案格式
predictions = [{"answer": ans} for ans in predicted_answers]

# 计算指标
vqa_metric = evaluate.load("vqa")
metrics = vqa_metric.compute(
    predictions=predictions,
    references=references
)

print(f"VQA 准确率: {metrics['overall']['accuracy']*100:.2f}%")

  0%|          | 0/2680 [00:00<?, ?it/s]

You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to 

KeyError: 'answer'

In [ ]:
vqa_metric = evaluate.load("vqa")
metrics = vqa_metric.compute(
    predictions=formatted_predictions,
    references=references
)

print(f"VQA 准确率: {metrics['overall']['accuracy']*100:.2f}%")
print(f"是/否类准确率: {metrics['yes/no']['accuracy']*100:.2f}%")
print(f"数字类准确率: {metrics['number']['accuracy']*100:.2f}%")
print(f"其他类准确率: {metrics['other']['accuracy']*100:.2f}%")

In [8]:
import torch
from PIL import Image
import requests
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration

# 指定使用的模型ID，这里以 google/paligemma-3b-mix-224 为例
model_id = "google/paligemma-3b-mix-224"

# 加载模型与处理器
model = PaliGemmaForConditionalGeneration.from_pretrained(model_id)
processor = AutoProcessor.from_pretrained(model_id)

# 设置设备（可选，如果有GPU则使用GPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

PaliGemmaForConditionalGeneration(
  (vision_tower): SiglipVisionModel(
    (vision_model): SiglipVisionTransformer(
      (embeddings): SiglipVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
        (position_embedding): Embedding(256, 1152)
      )
      (encoder): SiglipEncoder(
        (layers): ModuleList(
          (0-26): 27 x SiglipEncoderLayer(
            (self_attn): SiglipSdpaAttention(
              (k_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (v_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (q_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (out_proj): Linear(in_features=1152, out_features=1152, bias=True)
            )
            (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
            (mlp): SiglipMLP(
              (activation_fn): PytorchGELUTanh()
              (fc1): Linear(in_features

In [26]:
image_file = "https://www.ilankelman.org/stopsigns/australia.jpg"
raw_image = Image.open(requests.get(image_file, stream=True).raw)


In [28]:
prompt = "What does this traffic sign instruct drivers to do? Answer:"
prompt_with_token = "<image> " + prompt
# 使用处理器对文本和图像进行预处理，得到模型输入
inputs = processor(raw_image, prompt_with_token, return_tensors="pt").to(device)
with torch.no_grad():
  output = model.generate(**inputs, max_new_tokens=20)

result = processor.decode(output[0], skip_special_tokens=True)
# 使用分隔符"Answer:"分割并提取后半部分，然后去除多余空格
answer = result.split("Answer:")[-1].strip()
answer

'stop'

You can find steps to infer [here](https://colab.research.google.com/drive/100IQcvMvGm9y--oelbLfI__eHCoz5Ser?usp=sharing).